In [102]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [103]:
load_dotenv()
openai = OpenAI()
claude = anthropic.Anthropic()

#OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"

# a bit on costlier side
OPENAI_MODEL = "gpt-4o"
#CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [104]:
def get_user_prompt(input_code, target_language):
    user_prompt = f"Rewrite given code to performance efficient code in {target_language} language using fastest possible implementation."
    user_prompt += "It should produce same output as given code."
    user_prompt += "Pay attention to data types. The code should be ready to execute with all necessary imports."
    user_prompt += input_code
    return user_prompt
    

In [105]:
system_message = "You are an assistant that reimplements given code in high performance expected target language for an M1 Mac. You are smart and technical coder who knows all well known programming language enough to identify launguage of given source code"
system_message += "Respond only with code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The converted code needs to produce an identical output in the fastest possible time."

In [106]:
def get_messages(input_code, target_language):
    return [{"role": "system", "content": system_message},
           {"role": "user", "content": get_user_prompt(input_code, target_language)}]

In [107]:
def write_output_to_file(content, target_language):
    if target_language == 'c++':
        code = content.replace("```cpp","").replace("```","")
        with open("c_optimized.cpp", "w") as f:
            f.write(code)
    elif target_language == 'java':
         code = content.replace("```java","").replace("```","")
         with open("java_optimized.java", "w") as f:
            f.write(code)
    elif target_language == 'python':
         code = content.replace("```python","").replace("```","")
         with open("python_optimized.py", "w") as f:
            f.write(code)
    else:
        print("Supported launguages for conversion are Java, C++ and Python")

In [108]:
def gpt_code_generator(python_code, target_language): 
    msgs=get_messages(python_code, target_language)
    print(msgs)
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=msgs, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
    write_output_to_file(reply, target_language)

In [109]:
# Need to add anthropic credits to hit claude API

def claude_code_generator(input_code, source_language, target_language):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": get_user_prompt(input_code, source_language, target_language)}])
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
    write_output_to_file(reply, target_language)

In [110]:
fibo = """
import time
def fibonacci_recursive(n):
    if n <= 1:
        return n
    return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)

def print_fibonacci_recursive(count):
    print("Fibonacci Series (Recursive):")
    for i in range(count):
        print(fibonacci_recursive(i), end=" ")
    print() 

start_time = time.time()
fibonacci_recursive(10)
end_time = time.time()
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [111]:
fibo_java = """
public class Fibonacci {

    // Use memoization to improve the performance of the Fibonacci calculation
    private static long[] memo;

    public static long fibonacciRecursive(int n) {
        if (n <= 1) {
            return n;
        }
        if (memo[n] != 0) {
            return memo[n];
        }
        memo[n] = fibonacciRecursive(n - 1) + fibonacciRecursive(n - 2);
        return memo[n];
    }

    public static void printFibonacciRecursive(int count) {
        System.out.println("Fibonacci Series (Recursive):");
        for (int i = 0; i < count; i++) {
            System.out.print(fibonacciRecursive(i) + " ");
        }
        System.out.println();
    }

    public static void main(String[] args) {
        int count = 10;
        memo = new long[count + 1]; // Initialize memoization array
        long startTime = System.nanoTime();
        fibonacciRecursive(count);
        long endTime = System.nanoTime();
        System.out.format("Execution Time: %.6f seconds%n", (endTime - startTime) / 1e9);
    }
}
"""

In [40]:
exec(fibo)

Execution Time: 0.000019 seconds


In [9]:
test_code = """
name = "Krupa"
print(f"Hello: {name}")
"""

In [36]:
exec(test_code)

Hello: Krupa


In [112]:
gpt_code_generator(fibo, 'java')

[{'role': 'system', 'content': 'You are an assistant that reimplements given code in high performance expected target language for an M1 Mac. You are smart and technical coder who knows all well known programming language enough to identify launguage of given source codeRespond only with code; use comments sparingly and do not provide any explanation other than occasional comments. The converted code needs to produce an identical output in the fastest possible time.'}, {'role': 'user', 'content': 'Rewrite given code to performance efficient code in java language using fastest possible implementation.It should produce same output as given code.Pay attention to data types. The code should be ready to execute with all necessary imports.\nimport time\ndef fibonacci_recursive(n):\n    if n <= 1:\n        return n\n    return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)\n\ndef print_fibonacci_recursive(count):\n    print("Fibonacci Series (Recursive):")\n    for i in range(count):\n  

In [114]:
gpt_code_generator(fibo_java, 'python')

[{'role': 'system', 'content': 'You are an assistant that reimplements given code in high performance expected target language for an M1 Mac. You are smart and technical coder who knows all well known programming language enough to identify launguage of given source codeRespond only with code; use comments sparingly and do not provide any explanation other than occasional comments. The converted code needs to produce an identical output in the fastest possible time.'}, {'role': 'user', 'content': 'Rewrite given code to performance efficient code in python language using fastest possible implementation.It should produce same output as given code.Pay attention to data types. The code should be ready to execute with all necessary imports.\npublic class Fibonacci {\n\n    // Use memoization to improve the performance of the Fibonacci calculation\n    private static long[] memo;\n\n    public static long fibonacciRecursive(int n) {\n        if (n <= 1) {\n            return n;\n        }\n 